In [1]:
import numpy as np

In [2]:
v = np.arange(10)
v

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [7]:
v = np.array([[1,2],[3,4]])

In [13]:
v = np.ravel(v, order='C')

In [9]:
import collections

In [14]:
collections.Counter(v)

Counter({1: 1, 2: 1, 3: 1, 4: 1})

In [4]:
(v <= 4).astype('float32')

array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [5]:
np.random.binomial(1, 0.5, 3)

array([1, 0, 0])

In [8]:
def test(param1, param2=None):
    if param2 == None:
        print(param1)
    else: print(param2)

In [10]:
test(1,2)

2


In [16]:
from collections import Counter
import torch

In [17]:
a = list()
a.append(torch.tensor([1,2]))
a.append(torch.tensor([3,4]))

In [22]:
torch.to_numpy(torch.argmax(a[1]))

AttributeError: module 'torch' has no attribute 'to_numpy'

In [15]:
action_list = [1,1,1,1,2,2,2]
c = Counter(action_list)
float(c.most_common(1)[0][0]) == 1

True

In [27]:
b = torch.tensor([3,1,2])
b

tensor([3, 1, 2])

In [33]:
int(np.argmax(b))

0

In [16]:
class test():
    def __init__(self):
        self.a = 1
        self.auc = self.train()

    def train(self):
        self.a = 2
        print('ddd')
        b=3
        return b

In [17]:
t = test()

ddd


In [1]:
a = 1

In [2]:
def test():
    b = 2
    if a==1:
        return b
    return b+1

In [3]:
test()

2

In [7]:
s = np.array([0], dtype=np.float32)

In [8]:
print(s)

[0.]


In [10]:
LEFT = 0
UP = 1
RIGHT = 2
DOWN = 3

other_prob = 0

In [11]:
tmp = [[UP, DOWN, LEFT, RIGHT],
                   [DOWN, LEFT, RIGHT, UP],
                   [LEFT, RIGHT, UP, DOWN],
                   [RIGHT, UP, DOWN, LEFT]]
tmp_dirs = [[[-1, 0], [1, 0], [0, -1], [0, 1]],
                        [[1, 0], [0, -1], [0, 1], [-1, 0]],
                        [[0, -1], [0, 1], [-1, 0], [1, 0]],
                        [[0, 1], [-1, 0], [1, 0], [0, -1]]]
tmp_pros = [[1 ,other_prob, other_prob, other_prob],
                        [1, other_prob, other_prob, other_prob],
                        [1, other_prob, other_prob, other_prob],
                        [1, other_prob, other_prob, other_prob], ]

In [12]:
for acts, dirs, probs in zip(tmp, tmp_dirs, tmp_pros):
                P[s][acts[0]] = self._calculate_transition_prob(position, dirs, probs)

In [14]:
import numpy as np
import sys
from io import StringIO
from typing import Tuple
from gym.envs.toy_text.discrete import DiscreteEnv
import time
from scipy.spatial.distance import cityblock

LEFT = 0
UP = 1
RIGHT = 2
DOWN = 3


class Specs:
    def __init__(self, max_steps):
        self.max_episode_steps = max_steps


class GridCore(DiscreteEnv):
    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self, shape: Tuple[int] = (5, 10), start: Tuple[int] = (0, 0),
                 goal: Tuple[int] = (0, 9), max_steps: int = 100,
                 percentage_reward: bool = False, no_goal_rew: bool = False,
                 dense_reward: bool = False, numpy_state: bool = True,
                 xy_state: bool = False):
        try:
            self.shape = self._shape
        except AttributeError:
            self.shape = shape
        self.nS = np.prod(self.shape, dtype=int)  # type: int
        self.nA = 4
        self.start = start
        self.goal = goal
        self.max_steps = max_steps
        self._steps = 0
        self._pr = percentage_reward
        self._no_goal_rew = no_goal_rew
        self.total_steps = 0
        self._dr = dense_reward
        self.spec = Specs(max_steps)
        self._nps = numpy_state # 얘가 True면 state가 1차원 array,
        self.xy = xy_state # 얘가 True면 state가 2차원 array 인듯.

        P = self._init_transition_probability()

        # We always start in state (3, 0)
        if start is not None:
            isd = np.zeros(self.nS)
            isd[np.ravel_multi_index(start, self.shape)] = 1.0
        else:
            isd = np.ones(self.nS)
            isd[np.ravel_multi_index(goal, self.shape)] = 0.0
            for pit in self._pits:
                isd[pit] = 0.0
            isd *= 1 / (self.nS - len(self._pits) - 1)

        super(GridCore, self).__init__(self.nS, self.nA, P, isd)

    def step(self, a):
        self._steps += 1
        s, r, d, i = super(GridCore, self).step(a)
        if self.xy:
            self.s = np.ravel_multi_index(self.s, self.shape)
        if self._steps >= self.max_steps:
            d = True
            i['early'] = True
            i['needs_reset'] = True
        self.total_steps += 1
        if self._nps:
            return np.array([s], dtype=np.float32), r, d, i
        # for one hot state
        s = np.eye(self.observation_space.n)[s]
        return s, r, d, i

    def reset(self):
        self._steps = 0
        # print(self.isd)
        self.s = np.random.choice(range(len(self.isd)), p=self.isd)
        self.lastaction = None
        if self.xy:
            s = np.unravel_index(self.s, self.shape)
            if self._nps:
                return np.array([s], dtype=np.float32)
            else:
                return s
        if self._nps:
            return np.array([self.s], dtype=np.float32)
        # for one hot state
        # s = np.eye(self.observation_space.n)[self.s] # 이부분 수정함
        return self.s # step 함수에서는 onehot encoding 하는데 얘는 왜 안할까..?

    def _init_transition_probability(self):
        raise NotImplementedError

    def _check_bounds(self, coord):
        coord[0] = min(coord[0], self.shape[0] - 1)
        coord[0] = max(coord[0], 0)
        coord[1] = min(coord[1], self.shape[1] - 1)
        coord[1] = max(coord[1], 0)
        return coord

    def print_T(self):
        print(self.P[self.s])

    def map_output(self, s, pos):
        if self.s == s:
            output = " x "
        elif pos == self.goal:
            output = " T "
        else:
            output = " o "
        return output

    def map_control_output(self, s, pos):
        return self.map_output(s, pos)

    def map_with_inbetween_goal(self, s, pos, in_between_goal):
        return self.map_output(s, pos)

    def render(self, mode='human', close=False, in_control=None, in_between_goal=None):
        self._render(mode, close, in_control, in_between_goal)

    def _render(self, mode='human', close=False, in_control=None, in_between_goal=None):
        if close:
            return
        outfile = StringIO() if mode == 'ansi' else sys.stdout
        if mode == 'human':
            print('\033[2;0H')

        for s in range(self.nS):
            position = np.unravel_index(s, self.shape)
            # print(self.s)
            if in_control:
                output = self.map_control_output(s, position)
            elif in_between_goal:
                output = self.map_with_inbetween_goal(s, position, in_between_goal)
            else:
                output = self.map_output(s, position)
            if position[1] == 0:
                output = output.lstrip()
            if position[1] == self.shape[1] - 1:
                output = output.rstrip()
                output += "\n"
            outfile.write(output)
        outfile.write("\n")
        if mode == 'human':
            if in_control:
                time.sleep(0.2)
            else:
                time.sleep(0.05)


class FallEnv(GridCore):
    _pits = []

    def __init__(self, act_fail_prob: float = 1.0, **kwargs):
        self.afp = act_fail_prob
        super(FallEnv, self).__init__(**kwargs)

    def _calculate_transition_prob(self, current, delta, prob):
        transitions = []
        for d, p in zip(delta, prob):
            new_position = np.array(current) + np.array(d)
            new_position = self._check_bounds(new_position).astype(int)
            new_state = np.ravel_multi_index(tuple(new_position), self.shape)
            if not self._dr:
                reward = -0.01
                is_done = False
                if tuple(new_position) == self.goal:
                    if self._pr:
                        reward = 1 - (self._steps / self.max_steps)
                    elif not self._no_goal_rew:
                        reward = 1.0
                    is_done = True
                elif new_state in self._pits:
                    reward = -1.
                    is_done = True
                transitions.append((p, new_position if self.xy else new_state, reward, is_done))
            else:
                reward = -cityblock(new_position, self.goal)
                is_done = False
                if tuple(new_position) == self.goal:
                    if self._pr:
                        reward = 1 - (self._steps / self.max_steps)
                    elif not self._no_goal_rew:
                        reward = 100.0
                    is_done = True
                elif new_state in self._pits:
                    reward = -100
                    is_done = True
                transitions.append((p, new_position if self.xy else new_state, reward, is_done))
        return transitions

    def _init_transition_probability(self):
        for idx, p in enumerate(self._pits):
            try:
                self._pits[idx] = np.ravel_multi_index(p, self.shape)
            except:
                pass  # <- this has to be here for the agent. Otherwise it throws an unexplainable error
        # Calculate transition probabilities
        P = {}
        for s in range(self.nS):
            position = np.unravel_index(s, self.shape)
            P[s] = {a: [] for a in range(self.nA)}
            other_prob = self.afp / 3.
            tmp = [[UP, DOWN, LEFT, RIGHT],
                   [DOWN, LEFT, RIGHT, UP],
                   [LEFT, RIGHT, UP, DOWN],
                   [RIGHT, UP, DOWN, LEFT]]
            tmp_dirs = [[[-1, 0], [1, 0], [0, -1], [0, 1]],
                        [[1, 0], [0, -1], [0, 1], [-1, 0]],
                        [[0, -1], [0, 1], [-1, 0], [1, 0]],
                        [[0, 1], [-1, 0], [1, 0], [0, -1]]]
            tmp_pros = [[1 - self.afp, other_prob, other_prob, other_prob],
                        [1 - self.afp, other_prob, other_prob, other_prob],
                        [1 - self.afp, other_prob, other_prob, other_prob],
                        [1 - self.afp, other_prob, other_prob, other_prob], ]
            for acts, dirs, probs in zip(tmp, tmp_dirs, tmp_pros):
                P[s][acts[0]] = self._calculate_transition_prob(position, dirs, probs)
        return P

    def map_output(self, s, pos):
        if self.s == s:
            output = " \u001b[33m*\u001b[37m "
        elif pos == self.goal:
            output = " \u001b[37mX\u001b[37m "
        elif s in self._pits:
            output = " \u001b[31m.\u001b[37m "
        else:
            output = " \u001b[30mo\u001b[37m "
        return output

    def map_control_output(self, s, pos):
        if self.s == s:
            return " \u001b[34m*\u001b[37m "
        else:
            return self.map_output(s, pos)

    def map_with_inbetween_goal(self, s, pos, in_between_goal):
        if s == in_between_goal:
            return " \u001b[34mx\u001b[37m "
        else:
            return self.map_output(s, pos)


class ZigZag6x10(FallEnv):
    _pits = [[0, 2], [0, 3],
             [1, 2], [1, 3],
             [2, 2], [2, 3],
             [3, 2], [3, 3],
             [5, 7], [5, 6],
             [4, 7], [4, 6],
             [3, 7], [3, 6],
             [2, 7], [2, 6],
             ]
    _shape = (6, 10)

In [15]:
import numpy as np

In [15]:
a = np.zeros(10)
b = np.zeros(10)

In [16]:
a[2] = 1
b[3] = 2

In [17]:
a

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
b

array([0., 0., 0., 2., 0., 0., 0., 0., 0., 0.])

In [19]:
np.where(a==True)[0][0]

2

In [21]:
aa = np.unravel_index(np.where(a==True)[0][0],(6,10))
bb = np.unravel_index(np.where(b==True)[0][0],(6,10))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [26]:
aa

(0, 2)

In [27]:
bb

(0, 3)

In [28]:
np.linalg.norm(aa-bb)

TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

In [35]:
np.linalg.norm(np.asarray(bb) - np.array([0,0]))

3.0

In [36]:
a = 1
b = 2

In [37]:
print(a==b)

False


In [38]:
c = a==b

In [40]:
a = np.array([1,2])
b = np.array([1,2])

In [42]:
np.concatenate((a,b),axis = 1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [43]:
import random

In [46]:
random.randint(0,1)

1

In [52]:
class test():
    def __init__(self, **kwargs):
        self.a = kwargs.get('c')
        self.b = kwargs.get('d')
    def test_print(self):
        print(self.a, self.b)

In [45]:
kwargs = dict()
kwargs['a']=1
kwargs['b']=2

kwargs

{'a': 1, 'b': 2}

In [54]:
test1 = test(c=1,d=1)

In [55]:
test.__init__(c=5,d=5)

TypeError: __init__() missing 1 required positional argument: 'self'

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

/home/sungbae/.local/share/virtualenvs/DSRL_PROJECT-4garSmRj/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [68]:
type(a)

list

In [72]:
b = np.array([1,2,3])
bb = []
for i in range(4):
    bb.append(b)

In [73]:
bb

[array([1, 2, 3]), array([1, 2, 3]), array([1, 2, 3]), array([1, 2, 3])]

In [84]:
a = [1,2,3]
aa = []
for i in range(3):
    aa.append(a)

In [85]:
aa

[[1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [77]:
cc = np.array(bb)

In [78]:
torch.tensor(cc, dtype=torch.float)

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.]])

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [87]:
class Q_network(nn.Module):
    
    def __init__(self, input_size):
        super(Q_network, self).__init__()

        self.input_size = input_size

        self.fc1 = nn.Linear(self.input_size, 3)
        self.fc2 = nn.Linear(3, 2)
        # self.fc3 = nn.Linear(5, 2)

    def forward(self, x):

        # x = torch.tensor(x, dtype=torch.float32)
        # x = torch.from_numpy(x)
        x = torch.tensor(x, dtype=torch.float32)
        x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        x = self.fc2(x)

        return x

In [80]:
print(device)

cpu


In [88]:
q = Q_network(3)

In [82]:
a

[1, 2, 3]

In [90]:
aa

[[1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [94]:
q(torch.tensor(aa))

/tmp/ipykernel_8639/296276274.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


tensor([[-0.2445,  0.6678],
        [-0.2445,  0.6678],
        [-0.2445,  0.6678]], grad_fn=<AddmmBackward0>)

In [95]:
d = np.array([1,2])

In [96]:
type(d)

numpy.ndarray

In [98]:
type(d) is np.ndarray

True